# Access FHIR server and query data in Python

NOTE: In order to access Monash FHIR server, you need to connect to monash VPN. 

To make HTTP request with *Python's Request Library*.
You need to install request with command "pip install requests" and import requests in the script.

In [58]:
import requests
import pandas as pd
from datetime import datetime

Provide **root url** for the server.

In [59]:
root_url = 'https://fhir.monash.edu/hapi-fhir-jpaserver/fhir/'

## Example 1

To get a **specific FHIR resource** from a FHIR server, extend the root url by appending the resource type you are looking for. e.g.[root_url]/Patient will return a bundle including all patients. 

In [60]:
patients_url = root_url +"Patient"

To get a particular Patient resource, you would want to use a Patient's id. Simply extend the patient_url by appending the patient's id.

In [61]:
patient_id_url = patients_url +"/1"

You can retrieve JSON data using HTTP request and use the builtin **JSON decoder** -json().

In [62]:
data = requests.get(url=patient_id_url).json()

In [63]:
data

{'resourceType': 'Patient',
 'id': '1',
 'meta': {'versionId': '1',
  'lastUpdated': '2020-03-17T22:28:33.774+11:00',
  'source': '#c0AZ8hIqSaQZCO5A'},
 'text': {'status': 'generated',
  'div': '<div xmlns="http://www.w3.org/1999/xhtml">Generated by <a href="https://github.com/synthetichealth/synthea">Synthea</a>.Version identifier: v2.2.0-350-g351054b\n .   Person seed: -4886276478734772288  Population seed: 1572249943013</div>'},
 'extension': [{'url': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-race',
   'extension': [{'url': 'ombCategory',
     'valueCoding': {'system': 'urn:oid:2.16.840.1.113883.6.238',
      'code': '2106-3',
      'display': 'White'}},
    {'url': 'text', 'valueString': 'White'}]},
  {'url': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-ethnicity',
   'extension': [{'url': 'ombCategory',
     'valueCoding': {'system': 'urn:oid:2.16.840.1.113883.6.238',
      'code': '2186-5',
      'display': 'Not Hispanic or Latino'}},
    {'url': 'text'

## Example 2 - Search with parameters

**Each resource type defines the parameters.** To search with parameters, we construct a URL starting with the root url followed by the resource type (e.g. Observation), a question mark character '?' and finished with the search parameters we wish to search, Patient=6430 in this case.

In [64]:
search1_url = root_url +"Observation?patient=6430"

To search on a particular observation, we can set the code for that observation. For exmaple, 'total cholesterol' has the the code '2093-3'. So when we want to get the patient's total cholesterol value, we can set 'code=2093-3'.

In [65]:
search2_url = root_url +"Observation?patient=6430&code=2093-3"

To manage returned resources, we can use paramter _sort to define the order of results and use _count to define how many results should be displayed in a single page.

NOTE: In order to keep the load on clidents, servers and the network minimized, the server may choose to return the results in a series of pages. The search result contains the URLs that the client uses to request additional pages from the search set. **The monash FHIR server return results in a series of pages. Each page contains 10 results as default.** Each page contains a URL to the previous page(if not the first page) and the next page(if not the last page). In this case, we use '_count=13' to display all results(13 results) in one page.

NOTE: _sort=date indicates that results are displayed in increasing order according to the issued date. _sort=-date indicates that results are displayed in decreasing order. 

In [66]:
search3_url= root_url +"Observation?patient=3689&code=2093-3&_sort=date&_count=13"

In [67]:
data2 = requests.get(url=search3_url).json()

In [68]:
data2

{'resourceType': 'Bundle',
 'id': '84d36543-0e50-4b0e-af1b-67fb2036952e',
 'meta': {'lastUpdated': '2020-04-09T12:15:12.286+10:00'},
 'type': 'searchset',
 'total': 4,
 'link': [{'relation': 'self',
   'url': 'https://fhir.monash.edu/hapi-fhir-jpaserver/fhir/Observation?_count=13&_sort=date&code=2093-3&patient=3689'}],
 'entry': [{'fullUrl': 'https://fhir.monash.edu/hapi-fhir-jpaserver/fhir/Observation/4048',
   'resource': {'resourceType': 'Observation',
    'id': '4048',
    'meta': {'versionId': '1',
     'lastUpdated': '2020-03-17T22:28:57.622+11:00',
     'source': '#fSYsD3HIhDgNEG4J'},
    'status': 'final',
    'category': [{'coding': [{'system': 'http://terminology.hl7.org/CodeSystem/observation-category',
        'code': 'laboratory',
        'display': 'laboratory'}]}],
    'code': {'coding': [{'system': 'http://loinc.org',
       'code': '2093-3',
       'display': 'Total Cholesterol'}],
     'text': 'Total Cholesterol'},
    'subject': {'reference': 'Patient/3689'},
    'en

### Extract data from json data in python

Json consists of attribute-value pairs and arrays. The above **data2** includes 7 attribute-value pairs: **resourceType, id, meta, type, total, link, entry**.
Set the attribute name using square brackets, then we can get the corresponding value. data2['entry'] gives an array in this case. 

In [69]:
entry=data2['entry']

In [70]:
len(entry)

4

In [71]:
Cholesterol_data = pd.DataFrame(columns =['cholestrol', 'issued'] )

In [72]:
for i in range(len(entry)):
        record=[]
        item = entry[i]['resource']
        weight = item['valueQuantity']['value']
        issued = item['issued']
        record.append(weight)
        record.append(issued)
        Cholesterol_data.loc[i] = record

In [73]:
Cholesterol_data

,cholestrol,issued
0,180.02,1999-08-24T15:48:33.327+10:00
1,178.88,2002-09-10T15:48:33.327+10:00
2,174.75,2005-09-27T15:48:33.327+10:00
3,175.00,2008-10-14T16:48:33.327+11:00


## Example 3  

In this example, Urls under 'link' attribute is used to load additional results. (As discussed above, the server will only return a single page with at most 10 records.)<br>
All patients' cholesterol values (if they are measured) as well as their basic information are recorded.

In [74]:
dReport_url= root_url +"DiagnosticReport"

In [75]:
data3 = pd.DataFrame(columns =['patientid','gender', 'birthDate',  'maritualStatus', 'totalCholesterol',"Triglycerides", 'lowDensity', 'highDensity', 'issued'] )

In [76]:
def checkDate(patient_id,new_date):
    # check whether the observation's issued date is the latest
    if patient_id not in data3.index:
        return True
    else:
        old_date = data3.loc[patient_id,'issued']
        if new_date > old_date:
            data3.drop([patient_id])
            return True
        else:
            return False

In [77]:
next_page = True
next_url = dReport_url
count_page = 0
count_patient = 0

while next_page == True:
    dReports = requests.get(url=next_url).json()
    
    # As discussed before, The monash FHIR server return results in a series of pages. 
    # Each page contains 10 results as default.
    # here we check and record the next page 
    next_page = False
    links = dReports['link']
    for i in range(len(links)):
        link=links[i]
        if link['relation'] == 'next':
            next_page = True
            next_url = link['url']
            count_page += 1
            
    # Extract data 
    entry = dReports['entry']
    for i in range(len(entry)):
        patient_array = []
        results = entry[i]['resource']['result']
        
        # Check whether this observation is on chterol or not.
        chterol = False
        for result in results:
            if result['display'] == 'Total Chterol':
                chterol = True
        
        # If this observation is on cholesterol value, then record the patient's id and issued date.
        if chterol == True:
            patient_id = entry[i]['resource']['subject']['reference'][len('Patient/'):]
            print(patient_id)
            issued = entry[i]['resource']['issued'][:len('2008-10-14')]
            date = datetime.strptime(issued, '%Y-%m-%d').date()

            # Get patient's basic information
            patient_data = requests.get(url = root_url+"Patient/"+patient_id).json()
            gender = patient_data['gender']
            birth = patient_data['birthDate']
            birthDate = datetime.strptime(birth, '%Y-%m-%d').date()
            maritalStatus = patient_data['maritalStatus']['text']
            
            check = checkDate(patient_id,date)
            
            # Check if the patient's Chterol value has already been recorded in the dataframe
            if check == True:
                count_patient+=1
                patient_array.append(patient_id)
                patient_array.append(gender)
                patient_array.append(birthDate)
                patient_array.append(maritalStatus)
                # Record chtoral(including total, Triglycerides, lowDensity and highDensity) value
                for result in results:
                    observation_ref = result['reference']
                    observation_data = requests.get(url = root_url + observation_ref).json()
                    value = observation_data['valueQuantity']['value']
                    patient_array.append(value)
                patient_array.append(date)
                data3.loc[patient_id] = patient_array 

253110
227754
227754
227754
227754
227754
227754
227754
227754
227754
227754
227754
227754
227754
253110
228575
228575
228575
229095
229095
229095
229396
229396
229396
230036
230036
230036
230036
230036
230036
230036
230036
230036
230527
230527
230527
230527
253936
253936
230823
230823
230823
230823
230823
230823
230823
230823
230823
253936
253936
253936
253936
253936
253936
253936
253936
253936
231355
231355
231355
253936
307879
232316
232316
232316
307879
232606
232606
232606
307879
256995
232708
232708
232708
233353
233353
233353
254593
254593
254593
254593
254593
254593
254593
254593
233762
233762
233762
233762
233762
233762
233762
233762
233762
233762
233762
233762
233762
233762
233762
233762
233762
233762
233762
233762
233762
233762
233762
254593
254593
254593
234624
234624
254593
234771
234771
234771
254593
254593
255558
255558
234937
234937
234937
234937
234937
234937
234937
234937
234937
234937
234937
234937
234937
234937
235573
235573
308258
235973
235973
235973
235973
235973

KeyboardInterrupt: 

In [ ]:
data3

Referrences(Good to look at): 
https://fhir-drills.github.io/simple-patient.html<br>
https://www.hl7.org/fhir/search.html